## project antenna pattern on the ellipsoid

In [ ]:
include("../modules/antenna.jl")
include("../modules/geometry.jl")
include("../modules/scene.jl")
using ReferenceFrameRotations
using LinearAlgebra
using Plots

# read in vpol grid
vgrid = Antenna.AntGrid("../inputs/darts_ant_03192021.nc");


# Create SC geometry based on primitives and project pattern on the surface

In [ ]:
#platform position and velocity
P_pos = Geometry.geo_to_xyz([0, 0, 500e3]); #platform position in ECEF
P_vel = [0, 0, 6e3]; #platform velocity

#look angle
θ_l = 0; 
side = "left";

#Sign for look angle side
lsgn(sd) = (sd == "l" || sd == "left") ? 1 : -1

# target volume grid on surface defined in geodetic lat, lon, height (θϕh)
t_θ=-2:.01:2 # deg
t_ϕ=-2:.005:2 # deg
t_h=0 # m target heights

#try the two different methods for creating this
@time t_geo_grid=Scene.form3Dgrid_for(t_θ,t_ϕ,t_h) # using 3 nested for loops 

#convert lat,lon,height grid to ECEF-xyz
t_xyz_grid=Geometry.geo_to_xyz(t_geo_grid) # convert target volume from geo to xyz


#get TCN frame to construct SC frame with
t,c,n = Geometry.get_tcn(P_pos, P_vel)

#create rotation quaternion from ECEF to tcn
q_ecef_tcn = dcm_to_quat(DCM([t';c';n']))
#create rotation quaternion from TCN to Ant frame
q_tcn_ant = Geometry.quat(-90, [0,0,1]) #rotate about nhat (in tcn) to match long side with track vector
q_ant_look = Geometry.quat(lsgn(side)*θ_l, [0,1,0]) #rotate about antenna y-hat (the long side) to the correct look angle 
q_ecef_ant = q_ecef_tcn* q_tcn_ant * q_ant_look 

#create look vector
lvec = t_xyz_grid .- P_pos;
lhat = lvec./norm.(eachcol(lvec))';

#project look vector onto the antenna frame
lhat_ant = zeros(3,size(lhat,2))
@time for ii=1:size(lhat,2)
    lhat_ant[:,ii] = Geometry.rotate_frame(lhat[:,ii], q_ecef_ant);
end

#interpolate antenna pattern
cp,xp = Antenna.interpolate_pattern(vgrid, lhat_ant);
cp = reshape(cp, length(t_ϕ), length(t_θ));
xp = reshape(xp, length(t_ϕ), length(t_θ));


# plot Magnitude
p1 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(cp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-copol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)
p2 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(xp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-crosspol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)

plot(p1, p2, layout = (1,2))


## Use SpaceCraft and antenna wrappers to project antenna patterns from the SimSetup module

In [ ]:
include("../modules/simsetup.jl")

P_pos = Geometry.geo_to_xyz([0, 0, 500e3]); #platform position in ECEF
P_vel = [0, 6e3, 0]; #platform velocity ECEF

#create antenna structure 
ant = SimSetup.sc_ant(vgrid); #additional arguments are rotation and origin 

#create spacecraft structure
sc = SimSetup.spacecraft(P_pos, P_vel, ant = ant, look_angle = 25, side = "right"); #ant, look_angle, side are optional

#inteprolate pattern
cp,xp = SimSetup.interpolate_pattern(sc, t_xyz_grid);

#reshape and plot
cp = reshape(cp, length(t_ϕ), length(t_θ));
xp = reshape(xp, length(t_ϕ), length(t_θ));


# plot Magnitude
p1 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(cp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-copol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)
p2 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(xp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-crosspol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)

plot(p1, p2, layout = (1,2))


## Rotate spacecraft using a quaterion and interpolate patterns

In [ ]:
P_vel = [0, 6e3, 0]

#create spacecraft structure
sc = SimSetup.spacecraft(P_pos, P_vel, ant = ant, look_angle = 0, side = side); #ant, look_angle, side are optional


#Roll spacecraft - this via quaternion
SimSetup.rotate_spacecraft!(sc, Geometry.quat(10, [0,0,1]))

#inteprolate pattern
cp,xp = SimSetup.interpolate_pattern(sc, t_xyz_grid);

#reshape and plot
cp = reshape(cp, length(t_ϕ), length(t_θ));
xp = reshape(xp, length(t_ϕ), length(t_θ));


# plot Magnitude
p1 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(cp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-copol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)
p2 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(xp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-crosspol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)

plot(p1, p2, layout = (1,2))


## Rotate spacecraft using rpy vectors and interpolate patterns

In [ ]:

P_vel = [0, 6e3, 0]

#create spacecraft structure
sc = SimSetup.spacecraft(P_pos, P_vel, ant = ant, look_angle = 10, side = side); #ant, look_angle, side are optional


#Roll spacecraft - via roll,pitch,yaw in that order (rpy are wrt tcn axes)
SimSetup.rotate_spacecraft!(sc, [10., 0., 10.]) #this is [r, p,y]

#inteprolate pattern
cp,xp = SimSetup.interpolate_pattern(sc, t_xyz_grid);

#reshape and plot
cp = reshape(cp, length(t_ϕ), length(t_θ));
xp = reshape(xp, length(t_ϕ), length(t_θ));


# plot Magnitude
p1 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(cp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-copol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)
p2 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(xp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-crosspol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)

plot(p1, p2, layout = (1,2))


## Rotate antenna using quaternion and interpolate patterns

In [ ]:
P_vel = [0, 6e3, 0]

#create spacecraft structure
sc = SimSetup.spacecraft(P_pos, P_vel, ant = ant, look_angle = 0, side = side); #ant, look_angle, side are optional


#Roll antenna - via quaternion
SimSetup.rotate_antenna!(sc.ant, Geometry.quat(10, [0,0,1]))

#inteprolate pattern
cp,xp = SimSetup.interpolate_pattern(sc, t_xyz_grid);

#reshape and plot
cp = reshape(cp, length(t_ϕ), length(t_θ));
xp = reshape(xp, length(t_ϕ), length(t_θ));


# plot Magnitude
p1 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(cp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-copol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)
p2 = heatmap(t_ϕ,t_θ, 20*log10.(abs.(xp')), 
    label = "", legend = :bottomleft, xlabel = "Longitude (deg)", ylabel = "Latitude (deg)",
    title = "V-crosspol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)

plot(p1, p2, layout = (1,2))

# create grid based on SCH and project patterns on it based on orbit data

In [ ]:
include("../modules/orbits.jl")
include("../modules/geometry.jl")
include("../modules/scene.jl")
include("../modules/simsetup.jl")

using NCDatasets
using Plots


#some parameters
look_angle = 50; #look angle in degrees
platfind = 1; #which platform to use
read_samps = 100; #how many samples to read from orbit file

#orbit file (note it should not be added to the repository)
ds = Dataset("../inputs/orbitOutput_082020.nc");

#read in the orbit position and velocity
pegind = Int(read_samps/2);
orbit_pos  = ds["position"][:,:,1:read_samps]*1e3;
orbit_vel  = ds["velocity"][:,:,1:read_samps]*1e3;
dcm = ds["dcm"];

#convert orbit position to ECEF
orbit_pos_ecef, orbit_vel_ecef = Orbits.ecef_orbitpos(orbit_pos, orbit_vel, dcm);

#convert orbit position to Geodetic Coords
platfgeo = Geometry.xyz_to_geo(orbit_pos_ecef[:,platfind,:]);

#create peg point. 1) compute heading, 2) create peg struct
hdg = Geometry.compute_heading(platfgeo[1,:]', platfgeo[2,:]', orbit_vel_ecef[:,platfind,:]);
peg = Geometry.PegPoint(platfgeo[1,pegind], platfgeo[2,pegind], hdg[1,pegind])

#create SCH grid
rs,rg = Scene.lookangle_to_range(look_angle, platfgeo[3,pegind], 0, peg.Ra)
s = -500e3:5e3:500e3;#along-track points
c = (-300e3:1e3:300e3) .+ rg; #cross-track points (note: +ve C assumes left looking geometry)
h = 0; #height - keep it on the surface
t_sch_grid = Geometry.sch_to_xyz(Scene.form3Dgrid_for(s, c, h), peg)

#create antenna structure 
ant = SimSetup.sc_ant(vgrid); #additional arguments are rotation and origin 

#create spacecraft structure
sc = SimSetup.spacecraft(orbit_pos_ecef[:,platfind,pegind], orbit_vel_ecef[:,platfind,pegind], ant = ant, 
    look_angle = look_angle, side = "left"); #ant, look_angle, side are optional but make sure this matches with C

#inteprolate pattern
cp,xp = SimSetup.interpolate_pattern(sc, t_sch_grid);

#reshape and plot
cp = reshape(cp, length(c), length(s));

# plot Magnitude
p1 = heatmap(s, c, 20*log10.(abs.(cp)), 
    label = "", legend = :bottomleft, ylabel = "Cross-track (deg)", xlabel = "Along-Track (deg)",
    title = "V-copol Grid Magnitude", clim=(-80,40), fill=true,aspect_ratio=:equal)
plot(p1)
